In [24]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [41]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [6]:
file_jan = "../data-homework/yellow_tripdata_2022-01.parquet"
file_feb = "../data-homework/yellow_tripdata_2022-02.parquet"

In [69]:
df_train = pd.read_parquet(file_jan)
df_val = pd.read_parquet(file_feb)

## Q1

In [70]:
df_train.shape

(2463931, 19)

## Q2

In [71]:
df_train["duration"] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train["duration"] = df_train["duration"].apply(lambda td: td.total_seconds() / 60)

In [72]:
df_train.duration.std()

46.44530513776802

## Q3

In [76]:
df_train_no_outliers = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]

In [77]:
1 - ((len(df_train) - len(df_train_no_outliers)) / len(df_train))

0.9827547930522404

## Q4

In [78]:
categorical = ['PULocationID', 'DOLocationID']
df_train_no_outliers[categorical] = df_train_no_outliers[categorical].astype(str)

dv = DictVectorizer()

train_dicts = df_train_no_outliers[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

/var/folders/lr/1y49n_xs1mng_5gz3c4824qh0000gn/T/ipykernel_40139/219972678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_no_outliers[categorical] = df_train_no_outliers[categorical].astype(str)


In [79]:
X_train.shape

(2421440, 515)

## Q5

In [80]:
target = "duration"
y_train = df_train_no_outliers[target].values

In [81]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190814952337

## Q6

In [43]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']

    df[categorical] = df[categorical].astype(str)

    return df

In [82]:
df_train = read_dataframe(file_jan)
df_val = read_dataframe(file_feb)

In [85]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [86]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [88]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786407163179794